In [1]:
from hyperpyyaml import load_hyperpyyaml
import speechbrain as sb
import logging
import os
import sys
from pathlib import Path
import boto3
import datetime
from tqdm import tqdm

In [2]:
##### download data from s3 storage
def get_s3_object_list(s3, bucket_name, prefix, max_keys):
    obj_list = []
    response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys, Prefix=prefix)

    while True:
        if response.get('Contents') is not None:

            for content in response.get('Contents'):
                filename = content.get('Key')
                date_info = content.get('LastModified')
                
                obj_list.append(filename)
        
            if response.get('IsTruncated'):
                response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys, Prefix=prefix,
                                        Marker=response.get('NextMarker'))
            else:
                break
        
        else:
            # logger.info(f'{prefix} : there is no data.')
            print(f'{prefix} : there is no data.')
            break
    
    return obj_list

In [3]:
service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
region_name = 'kr-standard'
access_key = '5CA6DF7F42B860BC5BF6'
secret_key = '521AFE59000AEEE6130AA408FD0D12E56D9AB1D7'

s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                aws_secret_access_key=secret_key)


In [16]:
bucket_name = 'aidata-2022-02-018'
prefix = '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/'
# prefix = '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기'
max_keys = 1000

# get_s3_object_list(s3, bucket_name, prefix, max_keys)

In [17]:
response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys, Prefix=prefix)

In [18]:
response.get('Contents')

[{'Key': '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/',
  'LastModified': datetime.datetime(2022, 10, 21, 8, 33, 18, 353000, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'Standard',
  'Owner': {'DisplayName': 'ncp-2632028-0', 'ID': 'ncp-2632028-0'}},
 {'Key': '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/',
  'LastModified': datetime.datetime(2022, 10, 21, 8, 33, 18, 352000, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'Size': 0,
  'StorageClass': 'Standard',
  'Owner': {'DisplayName': 'ncp-2632028-0', 'ID': 'ncp-2632028-0'}},
 {'Key': '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_0_5.wav',
  'LastModified': datetime.datetime(2023, 2, 24, 19, 46, 7, 708000, tzinfo=tzutc()),
  'ETag': '"8862919dc475e633f7885488ce525edf"',
  'Size': 177644,
  'StorageClass': 'Standard',
  'Owner': {'DisplayName': 'ncp-26

In [19]:
response[0]

KeyError: 0

In [20]:
file_names = []
folders = []

In [21]:
for content in response.get('Contents'):
    filename = content.get('Key')
    if filename[-1] == '/':
        folders.append(filename)
    else:
        file_names.append(filename)

In [22]:
folders

['139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/']

In [23]:
file_names

['139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_0_5.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_1_4.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_2_4.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_3_4.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgs440/st_set1_collectorgs440_speakergw1329_4_6.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgw101/st_set1_collectorgw101_speakergw1738_0_10.wav',
 '139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)/06.품질검증/1.Dataset/1.원천데이터/01. 강원도/01. 1인발화 따라말하기/collectorgw101/st_set1_collector

[1, 2, 3, 4, 5]